https://colab.research.google.com/drive/1gejQbVg0zSRXO4QXakYSb3yyaKJWnbQa

## Домашнее задание №3 
### Задача из курса _Python для решения практических задач_
#### Задача 3.1

В этой задаче нужно просто установить библиотеку xmltodict, скачать файл https://stepik.org/media/attachments/lesson/245571/map1.osm, создать в директории с файлом скрипт со следующим содержанием:

In [1]:
import xmltodict
from urllib.request import urlopen
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

if __name__ == '__main__':
    fin = urlopen(
        'https://stepik.org/media/attachments/lesson/245571/map1.osm')
    xml = fin.read().decode('utf-8')
    fin.close()

    parsedxml = xmltodict.parse(xml)
    print(parsedxml['osm']['node'][100]['@id'])

296932122


#### Задача 3.2
В OpenStreetMap XML встречаются теги node, которые соответствуют некоторым точкам на карте. Ноды могут не только обозначать какой-то точечный объект, но и входить в состав way (некоторой линии, возможно замкнутой) и не иметь собственных тегов. Для доступного по ссылке https://stepik.org/media/attachments/lesson/245678/map1.osm фрагмента карты посчитайте, сколько node имеет хотя бы один вложенный тэг tag, а сколько - не имеют. В качестве ответа введите два числа, разделённых пробелом.

In [2]:
file = urlopen(
    'https://stepik.org/media/attachments/lesson/245678/map1.osm').read()
xml = file.decode('utf-8')

parsed_xml = xmltodict.parse(xml)

tags = 0
for node in parsed_xml['osm']['node']:
    if 'tag' in node:
        tags += 1
no_tags = len(parsed_xml['osm']['node']) - tags
print(f'{tags} {no_tags}')

316 6166


#### Задача 3.3
Вася решил открыть АЗС (заправку). Чтобы оценить уровень конкуренции он хочет изучить количество заправок в интересующем его районе. Вася скачал интересующий его кусок карты OSM https://stepik.org/media/attachments/lesson/245681/map2.osm и хочет посчитать, сколько на нём отмечено точечных объектов (node), являющихся заправкой. В качестве ответа вам необходимо вывести одно число - количество АЗС.

"Как обозначается заправка в OpenStreetMap" - пример хорошего запроса чтобы узнать, как обозначается заправка в OpenStreetMap.

In [3]:
file = urlopen(
        'https://stepik.org/media/attachments/lesson/245681/map2.osm').read()

xml = file.decode('utf-8')
xml = xmltodict.parse(xml)
num = 0

for node in xml['osm']['node']:
    if 'tag' in node:
        if isinstance(node['tag'], list):
            for tag in node['tag']:
                if tag['@v'] == 'fuel':
                    num += 1
        elif node['tag']['@v'] == 'fuel':
            num += 1

print(num)

15


#### Задача 3.4
Вася решил открыть АЗС (заправку). Чтобы оценить уровень конкуренции он хочет изучить количество заправок в интересующем его районе. Вася скачал интересующий его кусок карты OSM https://stepik.org/media/attachments/lesson/245681/map2.osm и хочет посчитать, сколько на нём отмечено точечных объектов (node), являющихся заправкой. В качестве ответа вам необходимо вывести одно число - количество АЗС.

"Как обозначается заправка в OpenStreetMap" - пример хорошего запроса чтобы узнать, как обозначается заправка в OpenStreetMap.

In [4]:
import xmltodict
from urllib.request import urlopen
import ssl
ssl._create_default_https_context = ssl._create_unverified_context


def number_of_objects(xml_dict, obj_name, area_type='node'):
    result_num = 0
    for node in xml_dict['osm'][area_type]:
        if 'tag' in node:
            if isinstance(node['tag'], list):
                for tag in node['tag']:
                    if tag['@v'] == obj_name:
                        result_num += 1
            elif node['tag']['@v'] == obj_name:
                result_num += 1
    return result_num


if __name__ == '__main__':
    file = urlopen(
        'https://stepik.org/media/attachments/lesson/245681/map2.osm').read()
    xml = file.decode('utf-8')
    xml = xmltodict.parse(xml)
    num = 0

    for area_t in ['node', 'way']:
        num += number_of_objects(xml, 'fuel', area_t)

    print(num)

27


#### Задача НЕ со stepik 
Экспортируйте с OSM часть карты, на которой находитесь вы (или любую другую интересную вам)

_Я решила проанализировать количество автобусных остановок в моем родном городе, Североморске_

In [5]:
url = 'https://www.openstreetmap.org/api/0.6/map?bbox=33.4021%2C69.0642%2C33.448%2C69.0832'
xml = urlopen(url).read().decode('utf-8')
xml = xmltodict.parse(xml)

Воспользуюсь функцией, которую написала в рамках курса на степике:

In [6]:
print(number_of_objects(xml, 'bus_stop', area_type='node'))

21


#### Работа с xml с помощью BeautifulSoup

In [7]:
from bs4 import BeautifulSoup

In [8]:
soup = BeautifulSoup(urlopen(url).read().decode('utf-8'))

len(soup.find_all('tag', {'v': 'bus_stop'}))

21

### Задание из курса Python: основы и применение
#### Задача 3.4
Рассмотрим два HTML-документа A и B.
Из A можно перейти в B за один переход, если в A есть ссылка на B, т. е. внутри A есть тег `<a href="B">`, возможно с дополнительными параметрами внутри тега.
Из A можно перейти в B за два перехода если существует такой документ C, что из A в C можно перейти за один переход и из C в B можно перейти за один переход.

Вашей программе на вход подаются две строки, содержащие url двух документов A и B.
Выведите Yes, если из A в B можно перейти за два перехода, иначе выведите No.

Обратите внимание на то, что не все ссылки внутри HTML документа могут вести на существующие HTML документы.

In [9]:
import requests

inp1, inp2 = input().strip(), input().strip()

soup_a = BeautifulSoup(requests.get(inp1).text, 'html.parser')

flag = False
for a in soup_a.find_all('a'):
    soup = BeautifulSoup(requests.get(a['href']).text, 'html.parser')
    for a_inside in soup.find_all('a'):
        if a_inside['href'] == inp2:
            flag = True
print('Yes') if flag else print('No')

 https://stepic.org/media/attachments/lesson/24472/sample0.html
 https://stepic.org/media/attachments/lesson/24472/sample1.html


No


In [10]:
import re

def get_hrefs(string):
    return re.findall(r'href=[\'"]?([^\'" >]+)', string)
    
inp1, inp2 = input().strip(), input().strip()

flag = False
hrefs_a = get_hrefs(requests.get(inp1).text)
for href in hrefs_a:
    if inp2 in get_hrefs(requests.get(href).text):
        flag = True

print('Yes') if flag else print('No')

 https://stepic.org/media/attachments/lesson/24472/sample0.html
 https://stepic.org/media/attachments/lesson/24472/sample2.html


Yes


In [11]:
import requests
import re

inp = input().strip()
html = requests.get(inp).text
result = re.findall(r'<a\s.*?\s?href=[\'\"](?:\w+\:\/\/)?(\w[a-zA-Z0-9.-]+)', html)
result = list(set(result))
result.sort()

for r in result:
    print(r)

 http://pastebin.com/raw/7543p0ns


adworker.ru
banner.rbc.ru
biztorg.ru
blogi.rbc.ru
cnews.ru
consensus.rbc.ru
conv.rbc.ru
credit.rbc.ru
data.rbc.ru
dict.rbc.ru
drinktime.ru
events.cnews.ru
export.rbc.ru
finolymp.ru
gift.cnews.ru
graph.rbc.ru
magazine.rbc.ru
map.rbc.ru
marketing.rbc.ru
memori.ru
otc-pif.rbc.ru
otc-stock.rbc.ru
pda.rbc.ru
pogoda.rbc.ru
portfolio.rbc.ru
quote-otc.rbc.ru
quote.ru
raiting.rbc.ru
rating.rbc.ru
realty.rbc.ru
redir.rbc.ru
rss.rbc.ru
seminar.rbc.ru
spb.rbc.ru
sport.rbc.ru
static.feed.rbc.ru
stock.rbc.ru
style.rbc.ru
ta.rbc.ru
tata.ru
top.rbc.ru
top100.rambler.ru
turbo.ru
tv.rbc.ru
ug.rbc.ru
ulov-umov.ru
videoarchive.rbc.ru
www.5ballov.ru
www.armd.ru
www.autonews.ru
www.biztorg.ru
www.cnews.ru
www.conf.rbc.ru
www.event.rbc.ru
www.iglobe.ru
www.informer.ru
www.ivd.ru
www.liveinternet.ru
www.m-2.ru
www.nashidengi.ru
www.pochta.ru
www.quote.ru
www.quoterate.ru
www.quotetotal.ru
www.rbc.ru
www.rbc.ua
www.rbcdaily.ru
www.rbcinfosystems.com
www.rbcnews.com
www.rbctv.ru
www.refunder.ru
www.salon.ru
www